In [1]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [2]:
def writeText(text, path, mode = 'w'):     # function to work with text files
    with open (path, mode, encoding = 'utf-8') as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):          # function to work with Json files
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):          # function to work with csv files
    import csv
    with open (outputFile, "w", newline='', encoding = 'utf-8') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [3]:
def getTxt(path):                 # open txt file in read mode
    return open(path, 'r').read()

def getCsv(path, delim = ','):    # insert data of csv in lists 
    list_return = []
    with open (path, encoding = 'utf-8') as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [9]:
if platform.system() == 'Windows':           # platform specific paths for opening prep_out.csv
    prep_data = getCsv('..\\output\\prep_out.csv')
else:
    prep_data = getCsv('../output/prep_out.csv')

In [10]:
print(len(prep_data))     # get amount of lists (data sets)
print(prep_data[0]) # print for test purposes the 1141st list

3573
[['domestic', ' general', ' government', ' health', ' expenditure', ' capita', ' current', ' us', ' $', ' countries'], ['public', ' expenditure', ' health', ' domestic', ' sources', ' capita', ' expressed', ' current', ' us', ' dollars'], ['social', ' issues', ' politics', ' environment', ' national', ' accounts', ' health', ' systems', ' by', ' country', ' military', ' budget']]


## Word2Vec

In [11]:
def sumVect(list_vect, ind):
    try:
        for i, elem in enumerate(list_vect):
            if not elem:
                del list_vect[i]
        
        int_len_vect = len(list_vect[0])
        list_vect_sum = [0] * int_len_vect
        for vect in list_vect:
            for i, dim in enumerate(vect):
                list_vect_sum[i] += float(dim)
        return(list_vect_sum)
    except Exception as e:
        print(ind)
        print(list_vect)
        print(i)
        print(list_vect_sum)
        print(e)

In [12]:
def getVecList(list_in):
    failcnt = 0
    failelemcnt = 0
    str_log = 'Started at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S') + '\n'
    print('Started at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'),'\n')
    # List of Lists as input needed
    adress = 'http://word2vec.ai.wu.ac.at/googlenews/model?word='
#     adress = 'http://word2vec.ai.wu.ac.at/twitter27_200/model?word='
    list_return = [[],[]]
    list_failed = [] 
    
    for i, elem in enumerate(list_in):
        inner_feat, inner_label = [], []
        str_1 = '-------------------------------\n' + 'Element ' + str(i)
        #print(str_1)
        str_log += str_1 + '\n'
    
        for str_elem in elem[1]:
            elem[0].append(str_elem)
        del elem[1]
        
# name and desc ----------------------------------------
        for str_elem in elem[0]:
            word = str_elem.strip() 
            try:
                inner_feat.append([float(x) for x in requests.get((adress+word)).text.replace(' ','').replace('[','').replace(']','').split(',')])
            except:
                try:
                    inner_feat.append([float(x) for x in requests.get((adress+word.title())).text.replace(' ','').replace('[','').replace(']','').split(',')])
                except Exception as e:
                    str_2 = 'fail\n' + adress+(word) + '\n' + str(e) + '\n' + requests.get((adress+word)).text
                    #print(str_2)
                    str_log += str_2 + '\n'
                    inner_feat.append([])
                    list_failed.append(word)
                    failcnt += 1

# label ----------------------------------------
        for str_elem in elem[1]:
            if str_elem:
                word = str_elem.strip()
                inner_label.append(str_elem)

        
# create output        
        if (inner_feat and inner_label):
            list_return[0].append(sumVect(inner_feat, i))
            list_return[1].append(inner_label)
        else:
            str_log += 'fail\nEmpty List can not be appended!\n'
            failelemcnt += 1
            
        
        
    print('Failed words: ', failcnt)
    print('Failed Elements: ', failelemcnt)
    str_log += '\nFinished at:\n' + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    writeText(str_log, '../output/log.txt')    
    print('\nEnded at:')
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')) 
    return list_return, list_failed

In [13]:
vect_list = getVecList(prep_data[0:500])

Started at:
2018-11-07 14:03:37 

Failed words:  169
Failed Elements:  0

Ended at:
2018-11-07 14:07:32


In [14]:
len(vect_list[0][0])

500

In [15]:
features = vect_list[0][0]
labels = vect_list[0][1]
missing = vect_list[1]

In [16]:
if platform.system() == 'Windows':
    writeCsv(features, '..\\output\\feat.csv')
    writeCsv(labels, '..\\output\\labels.csv')
    writeCsv(missing, '..\\output\\missing_words.csv')
else:
    writeCsv(features, '../output/feat.csv')
    writeCsv(labels, '../output/labels.csv')
    writeCsv(missing, '../output/missing_words.csv')

In [12]:
print(features[0])
print('-----------------------------------')
print(labels[0])

[-0.4062042236328125, 0.378692626953125, 0.580841064453125, 2.324951171875, -2.383056640625, -1.06573486328125, 0.7054443359375, -1.952545166015625, 2.374755859375, -0.3231201171875, -1.4234619140625, -1.55322265625, -1.1827392578125, 1.4083251953125, -3.926025390625, 2.5245361328125, -0.515380859375, 1.62860107421875, -0.4010009765625, 1.962646484375, -0.6202392578125, -1.259765625, 0.23626708984375, -0.247314453125, 0.7353515625, 0.349609375, -2.01123046875, 1.974395751953125, -1.077392578125, -0.42816162109375, 0.087982177734375, -3.16644287109375, 0.10443115234375, -0.18048095703125, -0.605072021484375, -2.25469970703125, -0.1572113037109375, 1.4627685546875, 0.416290283203125, 1.8201904296875, 1.298828125, 0.150054931640625, 1.053466796875, -0.203369140625, -0.7011642456054688, -1.74127197265625, -0.5694580078125, 1.4952392578125, -3.080322265625, -0.504119873046875, 2.379547119140625, -1.0993804931640625, -0.5980224609375, 0.7698974609375, -1.5465087890625, -0.7574462890625, -2.6